# MXResNet.

> MXResNet model.

MXResNet model, [forum discussion](https://forums.fast.ai/t/how-we-beat-the-5-epoch-imagewoof-leaderboard-score-some-new-techniques-to-consider)

In [ ]:
#hide
from typing import List
from functools import partial

import torch
from torch import nn

from model_constructor import ModelConstructor
from model_constructor.xresnet import xresnet_stem, XResNet

## MXResNet constructor.

In [ ]:
mxresnet  = ModelConstructor(
    name='MXResNet',
    make_stem=xresnet_stem,
    stem_sizes=[3, 32, 64, 64],
    act_fn=torch.nn.Mish,
)

In [ ]:
mxresnet

MXResNet
  in_chans: 3, num_classes: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  act_fn: Mish, sa: False, se: False
  stem sizes: [3, 32, 64, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [ ]:
mxresnet.print_changed_fields()

Changed fields:
name: MXResNet
act_fn: Mish
stem_sizes: [3, 32, 64, 64]
make_stem: xresnet_stem


In [ ]:
#collapse_output
mxresnet.stem

Sequential(
  (conv_0): ConvBnAct(
    (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish(inplace=True)
  )
  (conv_1): ConvBnAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish(inplace=True)
  )
  (conv_2): ConvBnAct(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish(inplace=True)
  )
  (conv_3): ConvBnAct(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_

In [ ]:
#collapse_output
mxresnet.body

Sequential(
  (l_0): Sequential(
    (bl_0): BasicBlock(
      (convs): Sequential(
        (conv_0): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish(inplace=True)
        )
        (conv_1): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (act_fn): Mish(inplace=True)
    )
    (bl_1): BasicBlock(
      (convs): Sequential(
        (conv_0): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish(inplace=True)
        )
        (conv_1): ConvBnAct(
          (conv)

In [ ]:
#collapse_output
mxresnet.body

Sequential(
  (l_0): Sequential(
    (bl_0): BasicBlock(
      (convs): Sequential(
        (conv_0): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish(inplace=True)
        )
        (conv_1): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (act_fn): Mish(inplace=True)
    )
    (bl_1): BasicBlock(
      (convs): Sequential(
        (conv_0): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish(inplace=True)
        )
        (conv_1): ConvBnAct(
          (conv)

In [ ]:
#collapse_output
mxresnet.head

Sequential(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=512, out_features=1000, bias=True)
)

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = mxresnet()(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size"

torch.Size([16, 1000])


## MxResNet constructors

Lets create constructor class for MxResnet.

Base MxResNet inherit from XResNet.

In [ ]:
class MxResNet(XResNet):
    stem_sizes: list[int] = [3, 32, 64, 64]
    act_fn: type[nn.Module] = nn.Mish

MXResnet34 inherit from MXResnet.

In [ ]:
class MxResNet34(MxResNet):
    layers: list[int] = [3, 4, 6, 3]

MXResnet50 inherit from MXResnet34.

In [ ]:
class MxResNet50(MxResNet34):
    expansion: int = 4
    block_sizes: list[int] = [256, 512, 1024, 2048]

Now we can create constructor from class adn change model parameters during initialization or after.

In [ ]:
mc = MxResNet34(num_classes=10)
mc

MxResNet34
  in_chans: 3, num_classes: 10
  expansion: 1, groups: 1, dw: False, div_groups: None
  act_fn: Mish, sa: False, se: False
  stem sizes: [3, 32, 64, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [3, 4, 6, 3]

In [ ]:
mc = MxResNet50()
mc

MxResNet50
  in_chans: 3, num_classes: 1000
  expansion: 4, groups: 1, dw: False, div_groups: None
  act_fn: Mish, sa: False, se: False
  stem sizes: [3, 32, 64, 64], stride on 0
  body sizes [256, 512, 1024, 2048]
  layers: [3, 4, 6, 3]

To create model - call model constructor object or create model from constructor.

In [ ]:
model = mc()

In [ ]:
model = MxResNet34.create_model()

model_constructor
by ayasyrev